# Chess Neural Network

Building a neural network that learns to play chess based on my gameplay data.

## Goal
Train a move prediction model to create an AI that plays like me.




In [1]:
# Importing necessary libraries
import pandas as pd
import ast
import torch
import torch.nn as nn

## Step 1: Load Chess Data

Load cleaned data from csv file


In [2]:
df = pd.read_csv("/Users/riteshbhandari/Documents/GitHub/Chess-engine/bot/data-analysis/cleaned_data.csv")
df.head()

,game_id,moves,num_moves,first_move
0,123118274906,"[('white', 'e4', True), ('black', 'e6', False)...",90,e4
1,123118510404,"[('white', 'd4', False), ('black', 'c5', True)...",141,d4
2,123118790014,"[('white', 'e4', False), ('black', 'e5', True)...",46,e4
3,123158939328,"[('white', 'e4', False), ('black', 'd5', True)...",88,e4
4,123160166430,"[('white', 'e4', True), ('black', 'e5', False)...",110,e4


# Step 2: Pre-processing

2.1 Turning features to numerical values (moves.. etc) 

2.2 Encoding the features to be trained

In [3]:
# convert string representation of list to actual list
df["moves"] = df["moves"].apply(ast.literal_eval)

# saving all the moves to single list to be encoded 
every_move = []
for game in df["moves"]:
    for move in game:
        every_move.append(move[1])

every_move[:10]

['e4', 'e6', 'd4', 'Qh4', 'Nc3', 'f5', 'Nf3', 'Qe7', 'e5', 'Qb4']

In [4]:
# getting all the unique moves
unique_moves = set(every_move)  # just the unique moves
print("Number of different moves:", len(unique_moves))
print()

# give move a number
move_to_number = {}

# turning integer back to moves (for future use)
number_to_move= {}

for i, move in enumerate(unique_moves):
    move_to_number[move] = i
    number_to_move[i] = move

# turning all the numbers into integers
number_moves = []
for move in every_move:
    number_moves.append(move_to_number[move])

# first 10 moves
print("First 10 moves as numbers: ")
print(number_moves[:10])
print()

# first 10 original moves
print("First 10 original moves: ")
print(every_move[:10]) 

Number of different moves: 1927

First 10 moves as numbers: 
[830, 1014, 698, 1688, 851, 644, 417, 1251, 634, 73]

First 10 original moves: 
['e4', 'e6', 'd4', 'Qh4', 'Nc3', 'f5', 'Nf3', 'Qe7', 'e5', 'Qb4']


In [5]:
# Create colors per game
colors_per_game = []

for game in df["moves"]:
    game_colors = []
    for move in game:
        if move[0] == "white":
            game_colors.append(1)
        else:  # black
            game_colors.append(0)
    colors_per_game.append(game_colors)

# Add to DataFrame
df["colors"] = colors_per_game

# Check first row
print(df[["moves", "colors"]].iloc[0])


moves     [(white, e4, True), (black, e6, False), (white...
colors    [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, ...
Name: 0, dtype: object


In [6]:
# turn white or black to integers ( 1  = (White), 0 = (Black) )
teoriat_moves_per_game = []

for game in df["moves"]:
    game_teoriat = []
    for move in game:
        if move[2] == True:
            game_teoriat.append(1)
        else:
            game_teoriat.append(0)
    teoriat_moves_per_game.append(game_teoriat)

# Add to DataFrame
df["teoriat_moves"] = teoriat_moves_per_game

# Check first row
print(df[["moves", "teoriat_moves"]].iloc[0])

moves            [(white, e4, True), (black, e6, False), (white...
teoriat_moves    [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, ...
Name: 0, dtype: object


In [7]:
#combine all the features into a single list of tuples seperated by games 
combined_features_per_game = []

for game_idx in range(len(df)):
    game_combined = []
    moves = df["moves"].iloc[game_idx]
    colors = df["colors"].iloc[game_idx]
    teoriat = df["teoriat_moves"].iloc[game_idx]
    
    for i in range(len(moves)):
        # (color, move_as_integer, your_move)
        game_combined.append((
            colors[i],
            move_to_number[moves[i][1]],  # convert move to integer
            teoriat[i]
        ))
    # Append the combined features for the game
    combined_features_per_game.append(game_combined)

# Add to DataFrame
df["game_data"] = combined_features_per_game

# Check first game data
print(combined_features_per_game[0])

[(1, 830, 1), (0, 1014, 0), (1, 698, 1), (0, 1688, 0), (1, 851, 1), (0, 644, 0), (1, 417, 1), (0, 1251, 0), (1, 634, 1), (0, 73, 0), (1, 537, 1), (0, 476, 0), (1, 692, 1), (0, 13, 0), (1, 1581, 1), (0, 177, 0), (1, 821, 1), (0, 171, 0), (1, 249, 1), (0, 26, 0), (1, 1077, 1), (0, 907, 0), (1, 1854, 1), (0, 1304, 0), (1, 1383, 1), (0, 169, 0), (1, 4, 1), (0, 733, 0), (1, 1424, 1), (0, 142, 0), (1, 1077, 1), (0, 807, 0), (1, 480, 1), (0, 544, 0), (1, 1766, 1), (0, 547, 0), (1, 1663, 1), (0, 1416, 0), (1, 1623, 1), (0, 293, 0), (1, 427, 1), (0, 486, 0), (1, 911, 1), (0, 701, 0), (1, 1766, 1), (0, 89, 0), (1, 1123, 1), (0, 1073, 0), (1, 459, 1), (0, 1870, 0), (1, 676, 1), (0, 1412, 0), (1, 886, 1), (0, 171, 0), (1, 1465, 1), (0, 1553, 0), (1, 1766, 1), (0, 1187, 0), (1, 1623, 1), (0, 814, 0), (1, 1450, 1), (0, 848, 0), (1, 688, 1), (0, 1543, 0), (1, 1453, 1), (0, 248, 0), (1, 1602, 1), (0, 658, 0), (1, 1450, 1), (0, 248, 0), (1, 1602, 1), (0, 658, 0), (1, 1508, 1), (0, 256, 0), (1, 342, 1),

In [8]:
# lets drop the columns with unneccessary data 
df = df.drop(columns=["moves", "colors", "teoriat_moves", "first_move","num_moves"])
df.head()

,game_id,game_data
0,123118274906,"[(1, 830, 1), (0, 1014, 0), (1, 698, 1), (0, 1..."
1,123118510404,"[(1, 698, 0), (0, 1482, 1), (1, 1428, 0), (0, ..."
2,123118790014,"[(1, 830, 0), (0, 634, 1), (1, 417, 0), (0, 14..."
3,123158939328,"[(1, 830, 0), (0, 1428, 1), (1, 634, 0), (0, 8..."
4,123160166430,"[(1, 830, 1), (0, 634, 0), (1, 103, 1), (0, 58..."


### Step 2.2:  Embedded layer Encoding 
